In [119]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
from bs4 import BeautifulSoup
import requests
%matplotlib inline

## LOAD RESOURCES ##

In [120]:
path = 'resources/extended_dataset.parquet'
df = pd.read_parquet(path)
df.head(2)

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,...,Re-inspection,Community,Neighborhood,Ward,Precinct,Violation IDs,Nb Violations,Nb Violations of high severity,Nb Violations of medium severity,Nb Violations of low severity
0,2320519,SALAM RESTAURANT,SALAM RESTAURANT,2002822,Restaurant,High,4634-4636 N KEDZIE AVE,60625,2019-10-25,Complaint,...,True,ALBANY PARK,"NORTH PARK,ALBANY PARK",33,27,[None],0,0,0,0
1,2320509,TAQUERIA EL DORADO,TAQUERIA EL DORADO,2694960,Restaurant,High,2114 W LAWRENCE AVE,60625,2019-10-25,License,...,True,LINCOLN SQUARE,LINCOLN SQUARE,47,38,"[3, 5, 25, 50, 53, 53, 56, 58]",8,2,1,5


In [121]:
df_filtered = df[(df['Zip'] != 0) & (df['Zip'] != 60627) & (df['Zip'] != 60666)]

In [122]:
zip_codes = np.sort(df_filtered['Zip'].unique())
zip_codes

array([60148, 60501, 60559, 60601, 60602, 60603, 60604, 60605, 60606,
       60607, 60608, 60609, 60610, 60611, 60612, 60613, 60614, 60615,
       60616, 60617, 60618, 60619, 60620, 60621, 60622, 60623, 60624,
       60625, 60626, 60628, 60629, 60630, 60631, 60632, 60633, 60634,
       60636, 60637, 60638, 60639, 60640, 60641, 60642, 60643, 60644,
       60645, 60646, 60647, 60649, 60651, 60652, 60653, 60654, 60655,
       60656, 60657, 60659, 60660, 60661, 60707, 60827])

In [128]:
try:
    df_zip_codes = pd.read_parquet('resources/zip_infos.parquet')
except:
    population_densities = []
    diversity_indexes = []
    median_household_incomes = []
    per_capita_incomes = []
    wealth_indexes = []

    for i, zip_code in enumerate(zip_codes[1:]):
        URL = 'https://illinois.hometownlocator.com/zip-codes/data,zipcode,{}.cfm'.format(zip_code)
        r = requests.get(URL)
        page_body = r.text
        soup = BeautifulSoup(page_body, 'html.parser')
        all_tables = soup.find_all('table')
        for table in all_tables:
            all_tr = table.find_all('tr')
            for tr in all_tr:
                all_td = tr.find_all('td')
                if len(all_td) >= 2:
                    if all_td[0].text == 'Population Density':
                        population_density = all_td[1].text.replace(',', '.')
                        population_densities.append(population_density)
                    if all_td[0].text == 'Diversity Index1':
                        diversity_index = all_td[1].text
                        diversity_indexes.append(diversity_index)
                    if all_td[0].text == 'Median Household Income' and not all_td[1].text.startswith('#'):
                        median_household_income = all_td[1].text[1:].replace(',', '.')
                        if median_household_income != '':
                            median_household_incomes.append(median_household_income)
                    if all_td[0].text == 'Per Capita Income' and not all_td[1].text.startswith('#'):
                        per_capita_income = all_td[1].text[1:].replace(',', '.')
                        if per_capita_income != '':
                            per_capita_incomes.append(per_capita_income)
                    if all_td[0].text == 'Wealth Index3':
                        wealth_index = all_td[1].text
                        wealth_indexes.append(wealth_index)

    dict_ = {'Zip' : zip_codes[1:], 'Diversity index' : diversity_indexes, 'Median household income' : median_household_incomes, 'Per capita income' : per_capita_incomes, 'Wealth index' : wealth_indexes}
    df_zip_codes = pd.DataFrame(dict_).set_index('Zip')
    
df_zip_codes.head()

,Diversity index,Median household income,Per capita income,Wealth index
Zip,,,,
60501,81,52.613,19.943,56
60559,59,66.017,39.936,112
60601,60,112.683,91.269,120
60602,57,100.105,71.987,116
60603,61,123.197,84.558,174


In [129]:
df_zip_codes.to_parquet('resources/zip_infos.parquet')